In [30]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [315]:
with open('./res/lab1.json') as lab1_file:
    data = json.load(lab1_file)
chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

In [239]:
def get_priority(operation):
    priorities = [
        ['(', '[', 'АЭМ', 'Ф', 'IF'],
        [')', ']', ',', ';', 'THEN', 'ELSE'],
        ['=', 'goto'],
        ['||'],
        ['&&'],
        ['!'],
        ['<', '>', '<=', '>=', '==', '!='],
        ['+', '-'],
        ['*', '/'],
        ['^'],
        ['RETURN'],
        [':'],
        ['НФ', 'КФ']
    ]
    for index, row in priorities:
        if operation in row:
            return index
    return False

In [240]:
def get_token_name(token):
    global service_words, operations, separators, constants, identifiers
    codes = ['W', 'O', 'R', 'C', 'I']
    tables = [service_words, operations, separators, constants, identifiers]
    if token[0] in ['C', 'I']:
        return tables[codes.index(token[0])][token[1]]['value']
    return tables[codes.index(token[0])][token[1]]

In [241]:
def get_token_type(token):
    global service_words, operations, separators, constants, identifiers
    token_name = get_token_name(token)
    if token_name in service_words:
        return 'service_word'
    if token_name in operations:
        return 'operation'
    if token_name in separators:
        return 'separator'
    if token[0] == 'C':
        return constants[token[1]]['type']
    if token[0] == 'I':
        return identifiers[token[1]]['type']
    return False

In [317]:
stack = []
result = []
AEM_counter = None
F_counter = None
M_counter = 0

identifier_context = False
if_context = None
type_context = None
var_pool = []
var_type = None
func_context = False
arg_counter = 0

for token in chain:
    token_name = get_token_name(token)
    token_type = get_token_type(token)
    previous_stack = list(stack)
#     print(token, token_name, token_type)
    
    if token_type != 'separator' or token_name in ['(', ')', '[', ']', ',', ':', ';']:
        if token_type == 'identifier':
            identifier_context = True
            
        if token_name == 'return':
            stack.append('RETURN')
            
#         Var and func declaring
        if token_name in ['void', 'int', 'float', 'char']:
            type_context = 'type'
            var_type = token_name
            var_pool = []
            if func_context:
                arg_counter = 1 if arg_counter == 0 else arg_counter
            
        if type_context in ['type', 'type ids'] and token_type == 'identifier':
            type_context = 'type ids'
            var_pool.append(token_name)
#         also look at the ';' handling below
            
        if token_name == '}' and not if_context:
            result.append('КФ')
            func_context = False
            arg_counter = 0
            
                
#         Conditions handling
        if if_context == 'if () {} else':
            if token_name == '{':
                if_context = 'if () {} else {'
            else:
                if_context = 'if () {} else operator'
                
        if if_context == 'if () {}':
            if token_name == 'else':
                while stack and stack[-1] != 'IF':
                    result.append(stack.pop())
                M_counter += 1
                result.append('M' + str(M_counter) + ' БП М' + str(M_counter-1) + ':')
                if_context == 'if () {} else'
            else:
                while stack and stack[-1] != 'IF':
                    result.append(stack.pop())
                if stack and stack[-1] == 'IF':
                    stack.pop()
                M_counter += 1
                result.append('M' + str(M_counter) + ':')
                if_context = None
                
        if token_name == 'if':
            stack.append('IF')
            if_context = 'if'
            
        if if_context == 'if ()':
            if token_name == '{':
                if_context = 'if () {'
            else:
                if_context = 'if () operator'
                
        if token_name == '}': # also look at the ";" handling below
            if if_context == 'if () {':
                print('IT WASS!!')
                while stack and stack[-1] != 'IF':
                    result.append(stack.pop())
                if stack and stack[-1] == 'IF':
                    stack.pop()
                result.append('M' + str(M_counter) + ':')
                if_context = None
                    
            if if_context == 'if () {} else {':
                while stack and stack[-1] != 'IF':
                    result.append(stack.pop())
                if stack and stack[-1] == 'IF':
                    stack.pop()
                result.append('M' + str(M_counter) + ':')
                if_context = None
                    
        if token_name == 'else' and if_context == 'if () operator':
            while stack and stack[-1] != 'IF':
                result.append(stack.pop())
            M_counter += 1
            result.append('M' + str(M_counter) + ' БП М' + str(M_counter-1) + ':')
            if_context = 'if () {} else'
                    
                
#         ";" handling
        if token_name == ';':
            while stack and stack[-1] != 'IF' and stack[-1] != 'Ф':
                result.append(stack.pop())
            if if_context in ['if () operator', 'if () {} else operator']:
                stack.pop()
                result.append('M' + str(M_counter) + ':')
                if_context = None
            if type_context == 'type ids':
                result.append(str(len(var_pool)) + ' '+ var_type)
                type_context = None
            if func_context:
                arg_counter += 1

#         Ordinary expression handling
        if token_type in ['identifier', 'integer', 'string', 'real']:
            result.append(token_name)
            
        if token_type in ['operation'] or token_name in ['=', 'goto', ':']:
            token_name = 'БП' if token_name == 'goto' else token_name
            if not stack:
                stack.append(token_name)
            else:        
                while stack and get_priority(stack[-1]) > get_priority(token_name):
                    result.append(stack.pop())
                else:
                    stack.append(token_name)
                    
#         Brackets handling
        if token_name == '(' and not identifier_context:
            stack.append('(')
            
        if token_name == ')' and not (F_counter):
            while stack[-1] != '(':
                print(stack)
                result.append(stack.pop())
            stack.pop()
            if stack and stack[-1] == 'IF':
                if_context = 'if ()'
                M_counter += 1
                result.append('М' + str(M_counter) + ' УПЛ')
            if type_context == 'type ids':
                result.append(str(len(var_pool)) + ' ' + var_type)
            
#         Arrays handling
        if token_name == '[':
            stack.append('АЭМ')
            AEM_counter = 2
            
        if token_name == ',' and AEM_counter:
            while stack[-1] != 'АЭМ':
                result.append(stack.pop())
            AEM_counter += 1
            
        if token_name == ']':
            while stack[-1] != 'АЭМ':
                result.append(stack.pop())
            result.append(str(AEM_counter) + ' ' + stack.pop())
            AEM_counter = None
            
#         Functions handling
        if token_name == '(' and identifier_context:
            stack.append('Ф')
            F_counter = 1
            
            if type_context:
                func_context = True
                arg_counter = 0
                result.append('1 ' + str(var_type))
                type_context = None
            
        if token_name == ',' and F_counter:
            while stack[-1] != 'Ф':
                result.append(stack.pop())
            F_counter += 1
        
        if token_name == ')' and F_counter:
            F_counter += 1
            
            if type_context == 'type ids':
                result.append(str(len(var_pool)) + ' ' + var_type)
                type_context = None
                F_counter = ''    
            
            while stack[-1] != 'Ф':
                result.append(stack.pop())
            if not F_counter:
                stack[-1] = (str(arg_counter + 1) if arg_counter > 0 else '0') + ' НФ'
            result.append(str(F_counter) + stack.pop())
            F_counter = None

        print(str(token_name), Fore.WHITE + str(token_type) + Style.RESET_ALL)
        print('Stack:', previous_stack, '->', stack)
        print('Result:', result)
        print()
        
        if token_type != 'identifier':
            identifier_context = False
            
    
previous_stack = list(stack)
while stack:
    result.append(stack.pop())

print(Fore.RED + 'END' + Style.RESET_ALL)
print('Stack:', previous_stack, '->', stack)
print('Result:', result, Fore.BLUE)

void service_word
Stack: [] -> []
Result: []

func identifier
Stack: [] -> []
Result: ['func']

( separator
Stack: [] -> ['Ф']
Result: ['func', '1 void']

float service_word
Stack: ['Ф'] -> ['Ф']
Result: ['func', '1 void']

f1 identifier
Stack: ['Ф'] -> ['Ф']
Result: ['func', '1 void', 'f1']

; separator
Stack: ['Ф'] -> ['Ф']
Result: ['func', '1 void', 'f1', '1 float']

int service_word
Stack: ['Ф'] -> ['Ф']
Result: ['func', '1 void', 'f1', '1 float']

f2 identifier
Stack: ['Ф'] -> ['Ф']
Result: ['func', '1 void', 'f1', '1 float', 'f2']

) separator
Stack: ['Ф'] -> []
Result: ['func', '1 void', 'f1', '1 float', 'f2', '1 int', '3 НФ']

{ service_word
Stack: [] -> []
Result: ['func', '1 void', 'f1', '1 float', 'f2', '1 int', '3 НФ']

return service_word
Stack: [] -> ['RETURN']
Result: ['func', '1 void', 'f1', '1 float', 'f2', '1 int', '3 НФ']

0 integer
Stack: ['RETURN'] -> ['RETURN']
Result: ['func', '1 void', 'f1', '1 float', 'f2', '1 int', '3 НФ', '0']

; separator
Stack: ['RETURN'] -

In [316]:
data = {
    'rpn': result
}

with open('./res/lab2.json', 'w') as outfile:
    json.dump(data, outfile)